In [2]:
import pandas as pd

# Cargar el archivo CSV
csv_file = '../data/precp/completada/gauge_completada.csv'  # Cambia a la ruta correcta de tu archivo CSV
df = pd.read_csv(csv_file)

# Asignar las coordenadas X e Y de las estaciones
# Puedes modificar estos valores según las coordenadas reales de tus estaciones
# Asignar las coordenadas de las estaciones (latitud, longitud)
coords = {
    'chusis': (-79.983, -5.533), 
    'chalaco': (-80.616, -5.175), 
    'huamarca': (-80.617, -5.184), 
    'huancabamba': (-79.971, -5.195), 
    'miraflores': (-79.717, -5.250),
    # Agregar más estaciones si es necesario
}

# Generar el archivo .pcp para SWAT
with open('output.pcp', 'w') as f:
    # Primera línea: número de estaciones y sus coordenadas
    f.write(f"{len(coords)}")
    for station, (lat, lon) in coords.items():
        f.write(f" {lon:.6f} {lat:.6f}")
    f.write("\n")
    
    # Escribir las fechas y las precipitaciones diarias
    for index, row in df.iterrows():
        # Formato de fecha: AAAAMMDD
        date = pd.to_datetime(row['Fecha'])
        date_str = date.strftime('%Y%m%d')
        
        # Precipitaciones: cada valor de precipitación para cada estación
        f.write(date_str)
        for station in coords.keys():
            f.write(f" {row[station]:.2f}")
        f.write("\n")

print("Archivo .pcp generado correctamente")


Archivo .pcp generado correctamente


In [3]:
df.shape

(12420, 6)

In [8]:
import pandas as pd

# Cargar el archivo CSV con tus datos de precipitaciones
csv_file = '../data/precp/completada/gauge_completada.csv'   # Cambia esto por la ruta de tu archivo CSV real
df = pd.read_csv(csv_file)

# Lista de estaciones
stations = ['chusis', 'chalaco', 'huamarca', 'huancabamba', 'miraflores']

k=1
# Crear un archivo txt por cada estación
for station in stations:
    # Crear archivo .txt con el nombre de la estación
    with open(f'{station}.txt', 'w') as f:
        # Escribir la primera fila con las fechas
        for date in pd.to_datetime(df['Fecha']):
            if k == 1:
                f.write(date.strftime('%Y%m%d') + "\n")
            k = k+1
        # Escribir los valores de precipitación (solo la columna de la estación)
        for value in df[station]:
            f.write(f"{value:.1f}\n")  # Valores con un decimal, igual que en el ejemplo
    k = 1
print("Archivos .txt generados correctamente para cada estación.")


Archivos .txt generados correctamente para cada estación.


In [11]:
# Cargar el archivo CSV con tus datos de precipitaciones
csv_file = '../data/tmin/tmin_piura.csv'   # Cambia esto por la ruta de tu archivo CSV real
df = pd.read_csv(csv_file)

# Lista de estaciones
stations = ['chusis', 'chalaco', 'huamarca', 'huancabamba', 'miraflores']

k=1
# Crear un archivo txt por cada estación
for station in stations:
    # Crear archivo .txt con el nombre de la estación
    with open(f'{station}_tmin.txt', 'w') as f:
        # Escribir la primera fila con las fechas
        for date in pd.to_datetime(df['fecha']):
            if k == 1:
                f.write(date.strftime('%Y%m%d') + "\n")
            k = k+1
        # Escribir los valores de precipitación (solo la columna de la estación)
        for value in df[station]:
            f.write(f"{value:.1f}\n")  # Valores con un decimal, igual que en el ejemplo
    k = 1
print("Archivos .txt generados correctamente para cada estación.")

Archivos .txt generados correctamente para cada estación.


In [12]:
# Cargar el archivo CSV con tus datos de precipitaciones
csv_file = '../data/tmax/tmax_piura.csv'   # Cambia esto por la ruta de tu archivo CSV real
df = pd.read_csv(csv_file)

# Lista de estaciones
stations = ['chusis', 'chalaco', 'huamarca', 'huancabamba', 'miraflores']

k=1
# Crear un archivo txt por cada estación
for station in stations:
    # Crear archivo .txt con el nombre de la estación
    with open(f'{station}_tmax.txt', 'w') as f:
        # Escribir la primera fila con las fechas
        for date in pd.to_datetime(df['fecha']):
            if k == 1:
                f.write(date.strftime('%Y%m%d') + "\n")
            k = k+1
        # Escribir los valores de precipitación (solo la columna de la estación)
        for value in df[station]:
            f.write(f"{value:.1f}\n")  # Valores con un decimal, igual que en el ejemplo
    k = 1
print("Archivos .txt generados correctamente para cada estación.")

Archivos .txt generados correctamente para cada estación.


In [6]:
import pandas as pd
import numpy as np

# Cargar el archivo CSV con tus datos de precipitaciones
csv_file = '../data/precp/completada/gauge_completada.csv'   # Cambia esto por la ruta de tu archivo CSV real

# Load the provided daily precipitation data file
precipitation_data = pd.read_csv(csv_file, parse_dates=['Fecha'])

# Extracting the 'year' and 'month' from the 'Fecha' column
precipitation_data['year'] = precipitation_data['Fecha'].dt.year
precipitation_data['month'] = precipitation_data['Fecha'].dt.month

# Initialize a list to hold results for all stations
results = []

# Loop through each station (excluding 'Fecha' column)
stations = precipitation_data.columns[1:-2]  # Adjust based on actual column names

for station in stations:
    station_data = precipitation_data[['year', 'month', station]].copy()
    station_data = station_data.rename(columns={station: 'precipitation'})
    
    # Calculate the monthly statistics for each year
    monthly_stats = station_data.groupby('month')['precipitation'].agg(
        pcp_ave='mean',
        pcp_sd='std',
        pcp_skew=lambda x: x.skew(),
        wet_dry=lambda x: ((x > 0) & (x.shift(1) == 0)).mean(),
        wet_wet=lambda x: ((x > 0) & (x.shift(1) > 0)).mean(),
        pcp_days=lambda x: (x > 0).sum(),
        pcp_hhr='sum'
    ).reset_index()

    monthly_stats['station'] = station  # Add the station name to the DataFrame
    results.append(monthly_stats)

# Combine all results into a single DataFrame
final_results = pd.concat(results, ignore_index=True)

# Display the final results
#import ace_tools as tools; tools.display_dataframe_to_user(name="Monthly Precipitation Averages and Stats", dataframe=final_results)



In [7]:
final_results

,month,pcp_ave,pcp_sd,pcp_skew,wet_dry,wet_wet,pcp_days,pcp_hhr,station
0,1,0.040446,0.258645,10.787874,0.039848,0.173624,225,42.63,chusis
1,2,0.225062,1.260318,8.135501,0.056133,0.137214,187,216.51,chusis
2,3,0.337600,2.136795,12.409230,0.055028,0.119545,197,355.83,chusis
3,4,0.200500,1.730604,19.436611,0.049020,0.114706,176,204.51,chusis
4,5,0.016186,0.163092,14.673076,0.024668,0.106262,149,17.06,chusis
5,6,0.003945,0.063905,15.856531,0.009804,0.090196,104,4.02,chusis
6,7,0.003017,0.035499,17.353073,0.016129,0.141366,168,3.18,chusis
7,8,0.001423,0.005965,7.823346,0.005693,0.145161,159,1.50,chusis
8,9,0.002650,0.032810,28.027890,0.011765,0.177451,197,2.70,chusis
9,10,0.011357,0.180051,24.564929,0.018027,0.116698,147,11.97,chusis


In [9]:
# Correcting the column name for date parsing and calculations
tmax_data = pd.read_csv('../data/tmax/tmax_piura.csv', parse_dates=['fecha'])
tmin_data = pd.read_csv('../data/tmin/tmin_piura.csv', parse_dates=['fecha'])

# Extracting 'year' and 'month' from the 'fecha' column for both datasets
tmax_data['year'] = tmax_data['fecha'].dt.year
tmax_data['month'] = tmax_data['fecha'].dt.month

tmin_data['year'] = tmin_data['fecha'].dt.year
tmin_data['month'] = tmin_data['fecha'].dt.month

# Initialize lists to hold results for max and min temperature calculations for each gauge station
results_max = []
results_min = []

# Loop through each station (excluding 'fecha', 'year', 'month' columns) for tmax
stations_max = tmax_data.columns[1:-2]  # Adjust based on actual column names

for station in stations_max:
    station_data_max = tmax_data[['year', 'month', station]].copy()
    station_data_max = station_data_max.rename(columns={station: 'temperature'})
    
    # Calculate the monthly statistics for the maximum temperature
    monthly_stats_max = station_data_max.groupby('month')['temperature'].agg(
        tmp_max_ave='mean',
        tmp_max_sd='std'
    ).reset_index()

    monthly_stats_max['station'] = station  # Add the station name to the DataFrame
    results_max.append(monthly_stats_max)

# Loop through each station (excluding 'fecha', 'year', 'month' columns) for tmin
stations_min = tmin_data.columns[1:-2]  # Adjust based on actual column names

for station in stations_min:
    station_data_min = tmin_data[['year', 'month', station]].copy()
    station_data_min = station_data_min.rename(columns={station: 'temperature'})
    
    # Calculate the monthly statistics for the minimum temperature
    monthly_stats_min = station_data_min.groupby('month')['temperature'].agg(
        tmp_min_ave='mean',
        tmp_min_sd='std'
    ).reset_index()

    monthly_stats_min['station'] = station  # Add the station name to the DataFrame
    results_min.append(monthly_stats_min)

# Combine all results into single DataFrames
final_results_max = pd.concat(results_max, ignore_index=True)
final_results_min = pd.concat(results_min, ignore_index=True)

# Merge the maximum and minimum temperature results based on month and station
final_results_combined = pd.merge(final_results_max, final_results_min, on=['month', 'station'])
final_results_combined

# Display the final combined results
#import ace_tools as tools; tools.display_dataframe_to_user(name="Monthly Temperature Averages and Stats", dataframe=final_results_combined)


,month,tmp_max_ave,tmp_max_sd,station,tmp_min_ave,tmp_min_sd
0,1,23.156341,0.811233,chalaco,14.186865,0.911623
1,2,23.076175,0.736878,chalaco,14.592285,0.705469
2,3,24.078163,0.580491,chalaco,14.332854,0.739475
3,4,24.008380,0.600183,chalaco,14.243768,0.553225
4,5,24.038416,0.619197,chalaco,14.236153,0.600331
5,6,23.765901,0.556923,chalaco,14.180098,0.541080
6,7,24.024532,0.641111,chalaco,13.089525,0.575619
7,8,24.379926,0.437000,chalaco,13.091606,0.528487
8,9,24.820513,0.582971,chalaco,13.132483,0.517391
9,10,24.902105,0.566224,chalaco,13.520350,0.581784


In [10]:
# Now merging the precipitation data with the temperature statistics based on 'station' and 'month'
merged_results = pd.merge(final_results, final_results_combined, on=['month', 'station'])

In [12]:
merged_results.to_csv('../data/precp/monthly.csv',index=False)